# avro vs. fastavro serialization performance
Testing speedup of the more limited fastavro module over the standard avro library with full functionality.

In [1]:
#!pip install avro-python3
#!pip install fastavro

In [2]:
import os
import sys
import io
import json
import fastavro
import avro.schema
import avro.io

## Writing data with a simple schema (non-nested)

Schema with no defaults set for nullable items.

In [3]:
simple = """{
	"namespace": "lsst.alert",
	"type": "record",
	"name": "diaSource",
	"fields": [
		{"name": "diaSourceId", "type": "long"},
		{"name": "ccdVisitId",  "type": "long"},
		{"name": "diaObjectId", "type": ["long", "null"]},
		{"name": "ssObjectId", "type": ["long", "null"]},
		{"name": "parentSourceId", "type": ["long", "null"]},
		{"name": "midPointTai", "type": "double"},
		{"name": "filterName", "type": "string"},
		{"name": "radec", "type": [{"type": "record",
			"name": "Radec", "fields": [
			{"name": "ra", "type": "double"},
			{"name": "dec", "type": "double"}
		]}]},
			{"name": "raSigma", "type": "float"},
			{"name": "decSigma", "type": "float"},
			{"name": "ra_dec_Cov", "type": "float"},
			{"name": "x", "type": "float"},
			{"name": "y", "type": "float"},
			{"name": "xSigma", "type": "float"},
			{"name": "ySigma", "type": "float"},
			{"name": "x_y_Cov", "type": "float"},
            {"name": "snr", "type": "float"},
		{"name": "psFlux", "type": "float"}
	]
}
"""

simple_schema = avro.schema.Parse(simple)
print("avro schema:\n %s \n" % simple_schema)

avro schema:
 {"type": "record", "name": "diaSource", "fields": [{"type": "long", "name": "diaSourceId"}, {"type": "long", "name": "ccdVisitId"}, {"type": ["long", "null"], "name": "diaObjectId"}, {"type": ["long", "null"], "name": "ssObjectId"}, {"type": ["long", "null"], "name": "parentSourceId"}, {"type": "double", "name": "midPointTai"}, {"type": "string", "name": "filterName"}, {"type": [{"type": "record", "name": "Radec", "fields": [{"type": "double", "name": "ra"}, {"type": "double", "name": "dec"}], "namespace": "lsst.alert"}], "name": "radec"}, {"type": "float", "name": "raSigma"}, {"type": "float", "name": "decSigma"}, {"type": "float", "name": "ra_dec_Cov"}, {"type": "float", "name": "x"}, {"type": "float", "name": "y"}, {"type": "float", "name": "xSigma"}, {"type": "float", "name": "ySigma"}, {"type": "float", "name": "x_y_Cov"}, {"type": "float", "name": "snr"}, {"type": "float", "name": "psFlux"}], "namespace": "lsst.alert"} 



Schema with default=null set for nullable items.

In [4]:
simple_null = """
{
	"namespace": "lsst.alert",
	"type": "record",
	"name": "diaSource",
	"fields": [
		{"name": "diaSourceId", "type": "long"},
		{"name": "ccdVisitId",  "type": "long"},
		{"name": "diaObjectId", "type": ["long", "null"], "default": null},
		{"name": "ssObjectId", "type": ["long", "null"], "default": null},
		{"name": "parentSourceId", "type": ["long", "null"] , "default": null},
		{"name": "midPointTai", "type": "double"},
		{"name": "filterName", "type": "string"},
		{"name": "radec", "type": [{"type": "record",
			"name": "Radec", "fields": [
			{"name": "ra", "type": "double"},
			{"name": "dec", "type": "double"}
		]}]},
			{"name": "raSigma", "type": "float"},
			{"name": "decSigma", "type": "float"},
			{"name": "ra_dec_Cov", "type": "float"},
			{"name": "x", "type": "float"},
			{"name": "y", "type": "float"},
			{"name": "xSigma", "type": "float"},
			{"name": "ySigma", "type": "float"},
			{"name": "x_y_Cov", "type": "float"},
		{"name": "snr", "type": "float"},
		{"name": "psFlux", "type": "float"}
	]
}
"""

simple_null_schema = avro.schema.Parse(simple_null)
print("avro schema with default = null \n %s \n" % simple_null_schema)

avro schema with default = null 
 {"type": "record", "name": "diaSource", "fields": [{"type": "long", "name": "diaSourceId"}, {"type": "long", "name": "ccdVisitId"}, {"type": ["long", "null"], "name": "diaObjectId", "default": null}, {"type": ["long", "null"], "name": "ssObjectId", "default": null}, {"type": ["long", "null"], "name": "parentSourceId", "default": null}, {"type": "double", "name": "midPointTai"}, {"type": "string", "name": "filterName"}, {"type": [{"type": "record", "name": "Radec", "fields": [{"type": "double", "name": "ra"}, {"type": "double", "name": "dec"}], "namespace": "lsst.alert"}], "name": "radec"}, {"type": "float", "name": "raSigma"}, {"type": "float", "name": "decSigma"}, {"type": "float", "name": "ra_dec_Cov"}, {"type": "float", "name": "x"}, {"type": "float", "name": "y"}, {"type": "float", "name": "xSigma"}, {"type": "float", "name": "ySigma"}, {"type": "float", "name": "x_y_Cov"}, {"type": "float", "name": "snr"}, {"type": "float", "name": "psFlux"}], "na

In [5]:
simple_data = {
	"diaSourceId":111111,
	"ccdVisitId":111111,
	"midPointTai":12314.142412,
	"filterName":"my favorite filter",
	"radec": {
		"ra": 124142.12414,
		"dec": 1214141.121
		},
	"raSigma": 5.1,
	"decSigma": 10.1,
	"ra_dec_Cov": 1.241,
	"x": 112.1,
	"y": 121.1,
	"xSigma": 1.2,
	"ySigma": 12.1,
	"x_y_Cov": 11.2,
	"snr": 41.1,
	"psFlux": 1241.0
}

print(sys.getsizeof(simple_data))

864


In [6]:
## avro functions

def write_avro_data(data_dict, avro_schema):
    """Encode json with avro module into avro format given a schema.
    """
    writer = avro.io.DatumWriter(avro_schema)
    bytes_io = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_io)
    writer.write(data_dict, encoder)
    return bytes_io


def read_avro_data(bytes_io, avro_schema):
    """Read avro data with avro module and decode with a given schema.
    """
    raw_bytes = bytes_io.getvalue()
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(avro_schema)
    message = reader.read(decoder)
    return message


## fastavro functions

def write_fastavro_data(data_dict, schema_json):
    """Encode json with fastavro module into avro format given a schema.
    """
    bytes_io = io.BytesIO()
    fastavro.schemaless_writer(bytes_io, schema_json, data_dict)
    return bytes_io


def read_fastavro_data(bytes_io, schema_json):
    """Read avro data with fastavro module and decode with a given schema.
    """
    bytes_io.seek(0)
    message = fastavro.schemaless_reader(bytes_io, schema_json)
    return message


### Encoding data with simple schemas

fastavro module needs defaults for all nullable items in order to parse schema for serializing data.  Use schemas with defaults set for both modules.

In [7]:
avro_data = write_avro_data(simple_data, simple_schema)
avro_null_data = write_avro_data(simple_data, simple_null_schema)

In [8]:
simple_null_json = simple_null_schema.to_json()
fastavro_null_data = write_fastavro_data(simple_data, simple_null_json)

### Decoding

In [9]:
avro_message = read_avro_data(avro_data, simple_schema)
print(avro_message)
print('avro data: %s' % sys.getsizeof(avro_data))

{'raSigma': 5.099999904632568, 'parentSourceId': None, 'midPointTai': 12314.142412, 'ccdVisitId': 111111, 'ySigma': 12.100000381469727, 'diaSourceId': 111111, 'xSigma': 1.2000000476837158, 'ssObjectId': None, 'snr': 41.099998474121094, 'filterName': 'my favorite filter', 'radec': {'ra': 124142.12414, 'dec': 1214141.121}, 'x_y_Cov': 11.199999809265137, 'x': 112.0999984741211, 'psFlux': 1241.0, 'y': 121.0999984741211, 'decSigma': 10.100000381469727, 'ra_dec_Cov': 1.2410000562667847, 'diaObjectId': None}
avro data: 214


In [10]:
avro_null_message = read_avro_data(avro_null_data, simple_null_schema)
print(avro_null_message)
print('avro null data: %s' % sys.getsizeof(avro_null_data))

{'raSigma': 5.099999904632568, 'parentSourceId': None, 'midPointTai': 12314.142412, 'ccdVisitId': 111111, 'ySigma': 12.100000381469727, 'diaSourceId': 111111, 'xSigma': 1.2000000476837158, 'ssObjectId': None, 'snr': 41.099998474121094, 'filterName': 'my favorite filter', 'radec': {'ra': 124142.12414, 'dec': 1214141.121}, 'x_y_Cov': 11.199999809265137, 'x': 112.0999984741211, 'psFlux': 1241.0, 'y': 121.0999984741211, 'decSigma': 10.100000381469727, 'ra_dec_Cov': 1.2410000562667847, 'diaObjectId': None}
avro null data: 214


In [11]:
fastavro_null_message = read_fastavro_data(fastavro_null_data, simple_null_json)
print(fastavro_null_message)
print('fastavro null data: %s' % sys.getsizeof(fastavro_null_data))

{'raSigma': 5.099999904632568, 'parentSourceId': None, 'midPointTai': 12314.142412, 'ccdVisitId': 111111, 'ySigma': 12.100000381469727, 'diaSourceId': 111111, 'xSigma': 1.2000000476837158, 'ssObjectId': None, 'snr': 41.099998474121094, 'filterName': 'my favorite filter', 'radec': {'ra': 124142.12414, 'dec': 1214141.121}, 'x_y_Cov': 11.199999809265137, 'x': 112.0999984741211, 'psFlux': 1241.0, 'y': 121.0999984741211, 'decSigma': 10.100000381469727, 'ra_dec_Cov': 1.2410000562667847, 'diaObjectId': None}
fastavro null data: 227


### timeit

In [12]:
writeAvroTime_simple = %timeit -o -n10000 (write_avro_data(simple_data, simple_null_schema)) 

10000 loops, best of 3: 199 µs per loop


In [13]:
readAvroTime_simple = %timeit -o -n10000 (read_avro_data(avro_null_data, simple_null_schema))

10000 loops, best of 3: 235 µs per loop


In [14]:
writeFastAvroTime_simple = %timeit -o -n10000 (write_fastavro_data(simple_data, simple_null_json))

10000 loops, best of 3: 40.6 µs per loop


In [15]:
readFastAvroTime_simple = %timeit -o -n10000 (read_fastavro_data(fastavro_null_data, simple_null_json))

10000 loops, best of 3: 24 µs per loop


## Writing alert schema, no postage stamp
The fastavro module cannot parse nested schema, so must use avro.schema.

In [16]:
schema_list = ['../schema/diasource.avsc', 
               '../schema/diaobject.avsc',
               '../schema/ssobject.avsc',
               '../schema/cutout.avsc',
               '../schema/alert.avsc']

data_file = '../data/alert.json'
with open(data_file) as f:
    alert_json = json.load(f)

In [17]:
def combine_schemas(schema_files):
    """Combine multiple nested schemas into a single schema.
    """
    known_schemas = avro.schema.Names()

    for s in schema_files:
        schema = load_single_avsc(s, known_schemas)
    return schema


def load_single_avsc(file_path, names):
    """Load a single avsc file.
    """
    with open(file_path) as file_text:
        json_data = json.load(file_text)
    schema = avro.schema.SchemaFromJSONData(json_data, names)
    return schema

The fastavro schemaless writer needs a json not an avro.schema type.  Use avro.schema to combine nested schemas and convert to json for fastavro.

In [18]:
alert_schema = combine_schemas(schema_list)
alert_schema_json = alert_schema.to_json()

### Encoding data with alert schemas


In [19]:
alert_avro_data = write_avro_data(alert_json, alert_schema)
print('avro data with alert schema: %s \n' % sys.getsizeof(alert_avro_data))

avro data with alert schema: 1388 



In [20]:
alert_fastavro_data = write_fastavro_data(alert_json, alert_schema_json)
print('fastavro data with alert schema: %s \n' % sys.getsizeof(alert_fastavro_data))

fastavro data with alert schema: 1304 



### Decoding

In [21]:
alert_avro_message = read_avro_data(alert_avro_data, alert_schema)
print(alert_avro_message)

{'l1dbId': 222222222, 'prv_diaSources': [{'iyy': None, 'apFluxErr': None, 'dipChi2': None, 'apFlux': None, 'dipDecl': None, 'trailRa': None, 'trailAngle': None, 'ra': 351.570546978, 'i_cov': None, 'flags': 0, 'dipAngle': None, 'psLnL': None, 'diaObjectId': None, 'dipFluxDiff': None, 'diaSourceId': 281323062375219198, 'totFlux': None, 'iyyPSF': None, 'filterName': 'r', 'trail_Cov': None, 'dipMeanFlux': None, 'totFluxErr': None, 'dip_Cov': None, 'fpBkgd': None, 'spuriousness': None, 'ixx': None, 'ps_Cov': None, 'ccdVisitId': 111111, 'dipLength': None, 'diffFlux': None, 'ixyPSF': None, 'trailNdata': None, 'trailDecl': None, 'trailLength': None, 'decl': 0.126243049656, 'ixxPSF': None, 'trailLnL': None, 'trailFlux': None, 'dipRa': None, 'midPointTai': 1480360995.0, 'x': 112.0999984741211, 'diffFluxErr': None, 'dipLnL': None, 'trailChi2': None, 'snr': 41.099998474121094, 'psRa': None, 'psFlux': 1241.0, 'ixy': None, 'parentDiaSourceId': None, 'fpBkgdErr': None, 'psNdata': None, 'ssObjectId': 

In [22]:
alert_fastavro_message = read_fastavro_data(alert_fastavro_data, alert_schema_json)
print(alert_fastavro_message)

{'l1dbId': 222222222, 'prv_diaSources': [{'iyy': None, 'apFluxErr': None, 'dipChi2': None, 'apFlux': None, 'dipDecl': None, 'trailRa': None, 'trailAngle': None, 'ra': 351.570546978, 'i_cov': None, 'flags': 0, 'dipAngle': None, 'psLnL': None, 'diaObjectId': None, 'dipFluxDiff': None, 'diaSourceId': 281323062375219198, 'totFlux': None, 'iyyPSF': None, 'filterName': 'r', 'trail_Cov': None, 'dipMeanFlux': None, 'totFluxErr': None, 'dip_Cov': None, 'fpBkgd': None, 'spuriousness': None, 'ixx': None, 'ps_Cov': None, 'ccdVisitId': 111111, 'dipLength': None, 'diffFlux': None, 'ixyPSF': None, 'trailNdata': None, 'trailDecl': None, 'trailLength': None, 'decl': 0.126243049656, 'ixxPSF': None, 'trailLnL': None, 'trailFlux': None, 'dipRa': None, 'midPointTai': 1480360995.0, 'x': 112.0999984741211, 'diffFluxErr': None, 'dipLnL': None, 'trailChi2': None, 'snr': 41.099998474121094, 'psRa': None, 'psFlux': 1241.0, 'ixy': None, 'parentDiaSourceId': None, 'fpBkgdErr': None, 'psNdata': None, 'ssObjectId': 

### timeit

In [23]:
writeAvroTime_alert = %timeit -o -n10000 (write_avro_data(alert_json, alert_schema)) 

10000 loops, best of 3: 8.37 ms per loop


In [24]:
readAvroTime_alert = %timeit -o -n10000 (read_avro_data(alert_avro_data, alert_schema))

10000 loops, best of 3: 7.07 ms per loop


In [25]:
writeFastAvroTime_alert = %timeit -o -n10000 (write_fastavro_data(alert_json, alert_schema_json))

10000 loops, best of 3: 1.66 ms per loop


In [26]:
readFastAvroTime_alert = %timeit -o -n10000 (read_fastavro_data(alert_fastavro_data, alert_schema_json))

10000 loops, best of 3: 572 µs per loop


## Writing alert schema, with postage stamp

In [27]:
cutoutdiff_path = 'stamp-676.fits'
cutouttemp_path = 'stamp-677.fits'


def load_stamp(file_path):
    """Load a cutout postage stamp file to include in alert.
    """
    _, fileoutname = os.path.split(file_path)
    with open(file_path, mode='rb') as f:
        cutout_data = f.read()
        cutout_dict = {"fileName": fileoutname, "stampData": cutout_data}
    return cutout_dict


def write_stamp_file(stamp_dict, output_dir):
    """Given a stamp dict that follows the cutout schema, write data to a file in a given directory.
    """
    filename = stamp_dict['fileName']
    try:
        os.makedirs(output_dir)
    except OSError:
        pass
    out_path = os.path.join(output_dir, filename)
    with open(out_path, 'wb') as f:
        f.write(stamp_dict['stampData'])
    return out_path


Prep stamp data.

In [28]:
cutoutDifference = load_stamp(cutoutdiff_path)
alert_json['cutoutDifference'] = cutoutDifference
cutoutTemplate = load_stamp(cutouttemp_path)
alert_json['cutoutTemplate'] = cutoutTemplate

### Encoding with stamps

In [29]:
alert_stamp_avro_data = write_avro_data(alert_json, alert_schema)
print('avro data with alert schema and stamp: %s \n' % sys.getsizeof(alert_stamp_avro_data))

avro data with alert schema and stamp: 81951 



In [30]:
alert_stamp_fastavro_data = write_fastavro_data(alert_json, alert_schema_json)
print('fastavro data with alert schema and stamp: %s \n' % sys.getsizeof(alert_stamp_fastavro_data))

fastavro data with alert schema and stamp: 81951 



### Decoding

In [31]:
alert_stamp_avro_message = read_avro_data(alert_stamp_avro_data, alert_schema)
message_text = {k: alert_stamp_avro_message[k] for k in alert_stamp_avro_message if k not in ['cutoutDifference', 'cutoutTemplate']}
print(message_text)
print('difference stamp exists: ', alert_stamp_avro_message['cutoutDifference'] != None)
print('template stamp exists: ', alert_stamp_avro_message['cutoutTemplate'] != None)

{'l1dbId': 222222222, 'prv_diaSources': [{'iyy': None, 'apFluxErr': None, 'dipChi2': None, 'apFlux': None, 'dipDecl': None, 'trailRa': None, 'trailAngle': None, 'ra': 351.570546978, 'i_cov': None, 'flags': 0, 'dipAngle': None, 'psLnL': None, 'diaObjectId': None, 'dipFluxDiff': None, 'diaSourceId': 281323062375219198, 'totFlux': None, 'iyyPSF': None, 'filterName': 'r', 'trail_Cov': None, 'dipMeanFlux': None, 'totFluxErr': None, 'dip_Cov': None, 'fpBkgd': None, 'spuriousness': None, 'ixx': None, 'ps_Cov': None, 'ccdVisitId': 111111, 'dipLength': None, 'diffFlux': None, 'ixyPSF': None, 'trailNdata': None, 'trailDecl': None, 'trailLength': None, 'decl': 0.126243049656, 'ixxPSF': None, 'trailLnL': None, 'trailFlux': None, 'dipRa': None, 'midPointTai': 1480360995.0, 'x': 112.0999984741211, 'diffFluxErr': None, 'dipLnL': None, 'trailChi2': None, 'snr': 41.099998474121094, 'psRa': None, 'psFlux': 1241.0, 'ixy': None, 'parentDiaSourceId': None, 'fpBkgdErr': None, 'psNdata': None, 'ssObjectId': 

In [32]:
alert_stamp_fastavro_message = read_fastavro_data(alert_stamp_fastavro_data, alert_schema_json)
message_text = {k: alert_stamp_fastavro_message[k] for k in alert_stamp_fastavro_message if k not in ['cutoutDifference', 'cutoutTemplate']}

print(message_text)
print('difference stamp exists: ', alert_stamp_fastavro_message['cutoutDifference'] != None)
print('template stamp exists: ', alert_stamp_fastavro_message['cutoutTemplate'] != None)

{'l1dbId': 222222222, 'prv_diaSources': [{'iyy': None, 'apFluxErr': None, 'dipChi2': None, 'apFlux': None, 'dipDecl': None, 'trailRa': None, 'trailAngle': None, 'ra': 351.570546978, 'i_cov': None, 'flags': 0, 'dipAngle': None, 'psLnL': None, 'diaObjectId': None, 'dipFluxDiff': None, 'diaSourceId': 281323062375219198, 'totFlux': None, 'iyyPSF': None, 'filterName': 'r', 'trail_Cov': None, 'dipMeanFlux': None, 'totFluxErr': None, 'dip_Cov': None, 'fpBkgd': None, 'spuriousness': None, 'ixx': None, 'ps_Cov': None, 'ccdVisitId': 111111, 'dipLength': None, 'diffFlux': None, 'ixyPSF': None, 'trailNdata': None, 'trailDecl': None, 'trailLength': None, 'decl': 0.126243049656, 'ixxPSF': None, 'trailLnL': None, 'trailFlux': None, 'dipRa': None, 'midPointTai': 1480360995.0, 'x': 112.0999984741211, 'diffFluxErr': None, 'dipLnL': None, 'trailChi2': None, 'snr': 41.099998474121094, 'psRa': None, 'psFlux': 1241.0, 'ixy': None, 'parentDiaSourceId': None, 'fpBkgdErr': None, 'psNdata': None, 'ssObjectId': 

### timeit

In [33]:
writeAvroTime_stamp = %timeit -o -n10000 (write_avro_data(alert_json, alert_schema)) 

10000 loops, best of 3: 7.99 ms per loop


In [34]:
readAvroTime_stamp = %timeit -o -n10000 (read_avro_data(alert_stamp_avro_data, alert_schema))

10000 loops, best of 3: 7.34 ms per loop


In [35]:
writeFastAvroTime_stamp = %timeit -o -n10000 (write_fastavro_data(alert_json, alert_schema_json))

10000 loops, best of 3: 1.84 ms per loop


In [36]:
readFastAvroTime_stamp = %timeit -o -n10000 (read_fastavro_data(alert_stamp_fastavro_data, alert_schema_json))

10000 loops, best of 3: 648 µs per loop


## Comparison

### Encoding

Small message and simple schema

In [37]:
print('avro messages serialized (alerts/sec): %d ' % (1/writeAvroTime_simple.best))
print('fastavro messages serialized (alerts/sec): %d ' % (1/writeFastAvroTime_simple.best))
print('fastavro speedup for simple schema: %0.2f' % (writeAvroTime_simple.best/writeFastAvroTime_simple.best))

avro messages serialized (alerts/sec): 5033 
fastavro messages serialized (alerts/sec): 24654 
fastavro speedup for simple schema: 4.90


Alert schema, no stamps

In [38]:
print('avro messages serialized (alerts/sec): %d ' % (1/writeAvroTime_alert.best))
print('fastavro messages serialized (alerts/sec): %d ' % (1/writeFastAvroTime_alert.best))
print('fastavro speedup for alert schema (no stamp): %0.2f' % (writeAvroTime_alert.best/writeFastAvroTime_alert.best))

avro messages serialized (alerts/sec): 119 
fastavro messages serialized (alerts/sec): 603 
fastavro speedup for alert schema (no stamp): 5.06


Alert schema, with stamps

In [39]:
print('avro messages serialized (alerts/sec): %d ' % (1/writeAvroTime_stamp.best))
print('fastavro messages serialized (alerts/sec): %d ' % (1/writeFastAvroTime_stamp.best))
print('fastavro speedup for alert schema (with stamp): %0.2f' % (writeAvroTime_stamp.best/writeFastAvroTime_stamp.best))

avro messages serialized (alerts/sec): 125 
fastavro messages serialized (alerts/sec): 542 
fastavro speedup for alert schema (with stamp): 4.33


### Decoding

Small message and simple schema

In [40]:
print('avro messages deserialized (alerts/sec): %d ' % (1/readAvroTime_simple.best))
print('fastavro messages deserialized (alerts/sec): %d ' % (1/readFastAvroTime_simple.best))
print('fastavro speedup for simple schema: %0.2f' % (readAvroTime_simple.best/readFastAvroTime_simple.best))

avro messages deserialized (alerts/sec): 4252 
fastavro messages deserialized (alerts/sec): 41689 
fastavro speedup for simple schema: 9.80


Alert schema, no stamps

In [41]:
print('avro messages deserialized (alerts/sec): %d ' % (1/readAvroTime_alert.best))
print('fastavro messages deserialized (alerts/sec): %d ' % (1/readFastAvroTime_alert.best))
print('fastavro speedup for alert schema (no stamp): %0.2f' % (readAvroTime_alert.best/readFastAvroTime_alert.best))

avro messages deserialized (alerts/sec): 141 
fastavro messages deserialized (alerts/sec): 1749 
fastavro speedup for alert schema (no stamp): 12.37


Alert schema, with stamps

In [42]:
print('avro messages deserialized (alerts/sec): %d ' % (1/readAvroTime_stamp.best))
print('fastavro messages deserialized (alerts/sec): %d ' % (1/readFastAvroTime_stamp.best))
print('fastavro speedup for alert schema (with stamp): %0.2f' % (readAvroTime_stamp.best/readFastAvroTime_stamp.best))

avro messages deserialized (alerts/sec): 136 
fastavro messages deserialized (alerts/sec): 1542 
fastavro speedup for alert schema (with stamp): 11.33
